In [1]:
import torchreid
model_name = 'resnet50'
dataset_name = 'market1501'
datamanager = torchreid.data.ImageDataManager(
    root="reid-data",
    sources=dataset_name,
    targets=dataset_name,
    height=256,
    width=128,
    batch_size_train=128,
    batch_size_test=100,
    transforms=["random_flip", "random_crop"]
)

/home/trewq/Desktop/Projects/RetailAnalytics/REID/RESEARCH/deep-person-reid/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gal

In [8]:
train_loader = datamanager.train_loader
for batch_idx, data in enumerate(train_loader):
    print(data['pid'])
    break

tensor([373, 367, 515, 327,  30, 465, 416, 111, 230, 684,  33, 151, 556, 627,
        229, 373, 190, 198, 395, 386, 499, 722, 520, 398, 682, 440,  55, 653,
        243,  54,  70,  61, 398, 425, 284, 397, 484, 526,  45, 531, 132, 352,
        373, 507, 217, 327, 224,  30,  77, 136, 127, 528,  48, 409, 577, 385,
        486,  63, 315, 684, 421, 151,  54, 132, 579, 175, 421,  57, 554, 186,
        708, 666, 165,  48, 519,  16, 612, 416, 227, 201, 613, 314,  70, 314,
        694,  96, 591,  92, 201, 327, 696, 244, 593,  93, 369, 490, 316, 443,
        342, 496, 343, 676, 104, 601, 537,  64, 167, 105, 474, 585, 440,  26,
        522, 708, 446, 114,  61, 173, 171, 601, 425, 580, 709, 463, 577, 459,
        399, 369])


In [4]:
model = torchreid.models.build_model(
    name=model_name,
    num_classes=datamanager.num_train_pids, #2000 for Tango
    loss="triplet",
    pretrained=True,
    pooling='gem',
    custom_layers=['ArcFaceMarginProduct']
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

In [5]:
# from tools.visualize_actmap import visactmap
# test_loader = datamanager.test_loader
# visactmap(model, test_loader, f"log/visactmap_{model_name}_{dataset_name}", 128, 256, True, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [7]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [8]:
engine.run(
    save_dir=f"log/{model_name}_{dataset_name}",
    max_epoch=60,
    eval_freq=5,
    print_freq=50,
    test_only=False,
    visrank=False,
    eval_metric='default'
)

=> Start training


  0%|          | 0/60 [00:00<?, ?it/s]

epoch: [1/60][50/101]	time 0.153 (0.156)	data 0.002 (0.006)	eta 0:15:39	loss 5.1502 (5.1459)	acc 100.0000 (100.0000)	lr 0.000000


  2%|▏         | 1/60 [00:15<15:22, 15.64s/it]

epoch: [1/60][100/101]	time 0.152 (0.154)	data 0.002 (0.004)	eta 0:15:20	loss 5.1373 (5.1428)	acc 100.0000 (100.0000)	lr 0.000000


...

 25%|██▌       | 15/60 [05:50<22:15, 29.67s/it]

Checkpoint saved to "log/resnet50_market1501/model/model.pth.tar-15"


epoch: [16/60][50/101]	time 0.152 (0.156)	data 0.002 (0.005)	eta 0:11:42	loss 5.0165 (5.0038)	acc 100.0000 (100.0000)	lr 0.000000


 25%|██▌       | 15/60 [05:59<17:59, 23.98s/it]


KeyboardInterrupt: 